In [1]:
%cd ../..

/home/batman/kaggle/kaggle-ashrae-DSR


In [5]:
from pathlib import Path
Path.cwd()

PosixPath('/home/batman/kaggle/kaggle-ashrae-DSR')

In [6]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 10 10:32:09 2018

@author: m.jones
"""

'\nCreated on Sun Jun 10 10:32:09 2018\n\n@author: m.jones\n'

In [7]:
CONTROL_PARAMS = {
    # The machine running, either LOCAL or KAGGLE
    # 'DEPLOYMENT': None,

    # The type of run:
    # SIMPLE
    # KERNEL (Default)
    # SEARCH
    # 'RUN_TYPE' : None,
}

In [8]:
# Logging
# =============================================================================
import sys
import logging
import datetime

logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.INFO)

# Create formatter
FORMAT = "%(asctime)s : %(message)s"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]

CONTROL_PARAMS['START_TIME'] = datetime.datetime.now()
logging.info("Logging started")

logging.info("Kernel started {}".format(CONTROL_PARAMS['START_TIME']))



2019-11-24 12:31:17 : Logging started
2019-11-24 12:31:17 : Kernel started 2019-11-24 12:31:17.108349


In [9]:
import os
from pathlib import Path

In [10]:
# Standard imports
# =============================================================================
import os
from pathlib import Path
import sys
import zipfile
import gc
import time
from pprint import pprint
from functools import reduce
from collections import defaultdict
import json
import yaml
import inspect
import gc
import random

In [11]:
# Basic imports
# =============================================================================
import tqdm

In [12]:
# ML imports
# =============================================================================
import numpy as np
print('numpy {} as np'.format(np.__version__))
import pandas as pd
print('pandas {} as pd'.format(pd.__version__))
from sklearn_pandas import DataFrameMapper
import sklearn as sk
print('sklearn {} as sk'.format(sk.__version__))

import sklearn.preprocessing
import sklearn.metrics
import sklearn.linear_model
import sklearn.pipeline
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.decomposition
import sklearn.compose
import sklearn.utils

# Models
import lightgbm as lgb
print("lightgbm", lgb.__version__)
import xgboost as xgb
print("xgboost", xgb.__version__)
# from catboost import CatBoostClassifier
import catboost as catb
print("catboost", catb.__version__)

numpy 1.17.4 as np
pandas 0.25.3 as pd
sklearn 0.21.3 as sk
lightgbm 2.3.0
xgboost 0.90
catboost 0.19.1


In [13]:
# Plotting
# =============================================================================
import matplotlib as mpl
print('matplotlib {} as mpl'.format(mpl.__version__))
import matplotlib.pyplot as plt
print('matplotlib.pyplot as plt'.format())
import seaborn as sns
print('seaboarn {} as sns'.format(sns.__version__))

# from IPython.core.display import display, HTML

# --- plotly ---
import plotly.io as pio
pio.renderers.default = "browser"
# from plotly import tools, subplots
import plotly.offline as py
# py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff



matplotlib 3.1.2 as mpl
matplotlib.pyplot as plt
seaboarn 0.9.0 as sns


In [14]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16 or not. feather format does not support float16.
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype



def get_building(df_meter_records, bldg_id):
    """
    Given the flat records for each meter-building-site

    Break out a summary energy profile DF for each building with the columns;
    Timestamp - Meter 1 - Meter 1 - Meter 1 - Meter 1
    Renames meter columns to proper energy sources
    """

    # Select the building records
    bldg_df = df_meter_records[df_meter_records.loc[:, 'building_id'] == bldg_id]
    bldg_df.drop('building_id', axis=1, inplace=True)
    logging.info("Building {} with {} records over {} meters".format(bldg_id, len(bldg_df), bldg_df['meter'].nunique()))

    # Pivot the records to meter-meter-meter-meter format
    bldg_df_pivot = bldg_df.pivot(index='timestamp', columns='meter', values='meter_reading')

    # For convenience, create empty columns for missing meters
    for meter_col in range(4):
        if meter_col not in bldg_df_pivot.columns:
            bldg_df_pivot[meter_col] = int(0)

    # bldg_df_pivot.memory_usage().sum() / 1024 ** 2 # Size in MB
    bldg_df_pivot.rename(columns={0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}, inplace=True)
    logging.info("{} to {}".format(bldg_df_pivot.index[0], bldg_df_pivot.index[-1]))
    assert bldg_df_pivot.index.is_monotonic_increasing

    return bldg_df_pivot

# for i in range(1000):
# bldg_id = i
# this_bldg = get_building(train_df, bldg_id)

In [15]:
def get_site_weather(weather_df, site_id):

    # Select the building records
    site_weather_df = weather_df[weather_df.loc[:, 'site_id'] == site_id]
    site_weather_df.drop('site_id', axis=1, inplace=True)
    logging.info("Site {} with {} records".format(site_id, len(site_weather_df)))
    site_weather_df.set_index('timestamp', drop=True, inplace=True)
    logging.info("{} to {}".format(site_weather_df.index[0], site_weather_df.index[-1]))
    assert site_weather_df.index.is_monotonic_increasing

    return site_weather_df

# site_id = 0
# for i in range(16):
#     r = get_site_weather(weather_train_df,i)
# r = weather_train_df.head()

In [16]:
def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    logging.debug('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    logging.debug('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    logging.info('Memory decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [19]:
class Map(dict):
    """
    Example:
    m = Map({'first_name': 'Eduardo'}, last_name='Pool', age=24, sports=['Soccer'])
    """
    def __init__(self, *args, **kwargs):
        super(Map, self).__init__(*args, **kwargs)
        for arg in args:
            if isinstance(arg, dict):
                for k, v in arg.iteritems():
                    self[k] = v

        if kwargs:
            for k, v in kwargs.iteritems():
                self[k] = v

    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

    def __setitem__(self, key, value):
        super(Map, self).__setitem__(key, value)
        self.__dict__.update({key: value})

    def __delattr__(self, item):
        self.__delitem__(item)

    def __delitem__(self, key):
        super(Map, self).__delitem__(key)
        del self.__dict__[key]

# Set the environment
SETTINGS = Map()
SETTINGS.data = Map()
SETTINGS.model = Map()

SETTINGS.data.path_data_root = Path.cwd() / 'data' / 'feather'
SETTINGS.data.use_ucf = True
SETTINGS.data.path_output = Path.cwd() / 'output'

SETTINGS.model.folds = 5
SETTINGS.model.num_rounds=1000
SETTINGS.control = Map()
SETTINGS.control.debug = False


logging.info("Settings:".format())
print(SETTINGS)

2019-11-24 12:31:50 : Settings:


{'data': {'path_data_root': PosixPath('/home/batman/kaggle/kaggle-ashrae-DSR/data/feather'), 'use_ucf': True, 'path_output': PosixPath('/home/batman/kaggle/kaggle-ashrae-DSR/output')}, 'model': {'folds': 5, 'num_rounds': 1000}, 'control': {'debug': False}}


In [20]:
logging.info(" *** Step 2: Load data *** ".format())

# Train
train_df = pd.read_feather(SETTINGS.data.path_data_root / 'train.feather')
logging.info("Loaded: train_df {} with {} buildings, {:0.1f} MB".format(train_df.shape, train_df.loc[:, 'building_id'].nunique(), train_df.memory_usage().sum() / 1024 ** 2))
r1 = train_df.head()
train_df = reduce_mem_usage(train_df)

# Test
test_df = pd.read_feather(SETTINGS.data.path_data_root / 'test.feather')
logging.info("Loaded: test_df {} with {} buildings, {:0.1f} MB".format(test_df.shape, test_df.loc[:, 'building_id'].nunique(), test_df.memory_usage().sum() / 1024 ** 2))
r2 = test_df.head()
test_df = reduce_mem_usage(test_df)

# Weather train
weather_train_df = pd.read_feather(SETTINGS.data.path_data_root/'weather_train.feather')
logging.info("Loaded: weather_train_df {}".format(weather_train_df.shape))
r = weather_train_df.head()
weather_train_df = reduce_mem_usage(weather_train_df)

# Weather test
weather_test_df = pd.read_feather(SETTINGS.data.path_data_root/'weather_test.feather')
logging.info("Loaded: weather_test_df {}".format(weather_test_df.shape))
weather_test_df = reduce_mem_usage(weather_test_df)

# Meta
building_meta_df = pd.read_feather(SETTINGS.data.path_data_root/'building_metadata.feather')
logging.info("Loaded: building_meta_df {}".format(building_meta_df.shape))
r = building_meta_df.head()
building_meta_df.set_index('building_id',inplace=True, drop=True)
building_meta_df = reduce_mem_usage(building_meta_df)

# Sample
sample_submission = pd.read_feather(os.path.join(SETTINGS.data.path_data_root, 'sample_submission.feather'))
logging.info("Loaded: sample_submission {}".format(sample_submission.shape))
sample_submission = reduce_mem_usage(sample_submission)

2019-11-24 12:31:51 :  *** Step 2: Load data *** 


ArrowIOError: Failed to open local file '/home/batman/kaggle/kaggle-ashrae-DSR/data/feather/train.feather', error: No such file or directory

In [ ]:
# train_merge = train_df_data.merge(building_meta_df, on='building_id', how='left')
# train_merge.memory_usage().sum() / 1024 ** 2
# test_merge = test_df_data.merge(building_meta_df, on='building_id', how='left')
# test_merge.memory_usage().sum() / 1024 ** 2
#
# train_df = train_merge.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
# train_df.memory_usage().sum() / 1024 ** 2
# r = train_df.head()
#
# test_df = test_merge.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')
# test_df.memory_usage().sum() / 1024 ** 2
# r = test_df.head()

# train2.info()

# del

In [ ]:
def preprocess_date_time_cols(df):
    df['date'] = df['timestamp'].dt.date
    logging.info("Added date column".format())
    df["hour"] = df["timestamp"].dt.hour
    logging.info("Added hour column".format())
    # df["day"] = df["timestamp"].dt.day
    df["weekend"] = df["timestamp"].dt.weekday
    logging.info("Added weekend column".format())
    df["month"] = df["timestamp"].dt.month
    logging.info("Added month column".format())
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    logging.info("Added dayofweek column".format())

    return df

In [ ]:
logging.info("Adding basic time features to train and test".format())
train_df = preprocess_date_time_cols(train_df)
test_df = preprocess_date_time_cols(test_df)

In [ ]:
train_df['meter_reading_log1p'] = np.log1p(train_df['meter_reading'])
test_df['meter_reading_log1p'] = np.log1p(test_df['meter_reading'])
logging.info("Added meter_reading_log1p [ln(1+x)] column".format())

In [ ]:
if 0:
    # test_df['building_mean'] = test_df['building_id'].map(building_mean)
    test_df['building_median'] = test_df['building_id'].map(building_median)
    # test_df['building_min'] = test_df['building_id'].map(building_min)
    # test_df['building_max'] = test_df['building_id'].map(building_max)
    # test_df['building_std'] = test_df['building_id'].map(building_std)

    print('preprocessing weather...')
    weather_test_df = weather_test_df.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))
    weather_test_df.groupby('site_id').apply(lambda group: group.isna().sum())

    # add_lag_feature(weather_test_df, window=3)
    # add_lag_feature(weather_test_df, window=72)

    print('reduce mem usage...')
    reduce_mem_usage(test_df, use_float16=True)
    reduce_mem_usage(weather_test_df, use_float16=True)

    gc.collect()



    logging.info("Removing building site=0".format())
    building_meta_df[building_meta_df.site_id == 0]
    train_df = train_df.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')


logging.info(" *** Step 3: Feature engineering *** ".format())

# Data preprocessing

Now, Let's try building GBDT (Gradient Boost Decision Tree) model to predict `meter_reading_log1p`. I will try using LightGBM in this notebook.

# Add time feature

Some features introduced in https://www.kaggle.com/ryches/simple-lgbm-solution by @ryches

Features that are likely predictive:

#### Weather

- time of day
- holiday
- weekend
- cloud_coverage + lags
- dew_temperature + lags
- precip_depth + lags
- sea_level_pressure + lags
- wind_direction + lags
- wind_speed + lags

#### Train

- max, mean, min, std of the specific building historically



However we should be careful of putting time feature, since we have only 1 year data in training,
including `date` makes overfiting to training data.

How about `month`? It may be better to check performance by cross validation.
I go not using this data in this kernel for robust modeling.



In [ ]:
# print(SETTINGS)
# sort train. i dont know it is best
# if use_ucf:
#     train_df = train_df.sort_values('month')
#     train_df = train_df.reset_index()

In [ ]:
logging.info("Adding building_median log1p feature".format())

df_group = train_df.groupby('building_id')['meter_reading_log1p']
#building_mean = df_group.mean().astype(np.float16)
building_median = df_group.median().astype(np.float16)
#building_min = df_group.min().astype(np.float16)
#building_max = df_group.max().astype(np.float16)
#building_std = df_group.std().astype(np.float16)

#train_df['building_mean'] = train_df['building_id'].map(building_mean)
train_df['building_median'] = train_df['building_id'].map(building_median)
#train_df['building_min'] = train_df['building_id'].map(building_min)
#train_df['building_max'] = train_df['building_id'].map(building_max)
#train_df['building_std'] = train_df['building_id'].map(building_std)
del df_group
logging.info("Added building_median log1p feature".format())

In [ ]:
#building_mean.head()

# Fill Nan value in weather dataframe by interpolation


weather data has a lot of NaNs!!

![](http://)I tried to fill these values by **interpolating** data.

In [ ]:
# weather_train_df.head()

In [ ]:
# weather_train_df.describe()

In [ ]:
# weather_train_df.isna().sum()

In [ ]:
# weather_train_df.shape

In [ ]:
logging.info("Interpolating over NaNs in weather_train_df".format())
# weather_train_df.groupby('site_id').apply(lambda group: group.isna().sum())
weather_train_df = weather_train_df.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [ ]:
# weather_train_df.groupby('site_id').apply(lambda group: group.isna().sum())

Seems number of nan has reduced by `interpolate` but some property has never appear in specific `site_id`, and nan remains for these features.

## lags

Adding some lag feature

In [ ]:
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [ ]:
# skip lag feature to save memory
#add_lag_feature(weather_train_df, window=3)
#add_lag_feature(weather_train_df, window=72)

In [ ]:
weather_train_df.head()

In [ ]:
# categorize primary_use column to reduce memory on merge...
logging.info("Convert primary_use to Categorical type".format())
primary_use_list = building_meta_df['primary_use'].unique()
primary_use_dict = {key: value for value, key in enumerate(primary_use_list)}
print('primary_use_dict: ', primary_use_dict)
building_meta_df['primary_use'] = building_meta_df['primary_use'].map(primary_use_dict)
gc.collect()

In [ ]:
reduce_mem_usage(train_df, use_float16=True)
reduce_mem_usage(building_meta_df, use_float16=True)
reduce_mem_usage(weather_train_df, use_float16=True)

In [ ]:
building_meta_df.head()

In [ ]:

category_cols = ['building_id', 'site_id', 'primary_use']  # , 'meter'
feature_cols = ['square_feet', 'year_built'] + [
    'hour', 'weekend',  # 'month' , 'dayofweek'
    'building_median'] + [
                   'air_temperature', 'cloud_coverage',
                   'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                   'wind_direction', 'wind_speed', ]


#     'air_temperature_mean_lag72',
#     'air_temperature_max_lag72', 'air_temperature_min_lag72',
#     'air_temperature_std_lag72', 'cloud_coverage_mean_lag72',
#     'dew_temperature_mean_lag72', 'precip_depth_1_hr_mean_lag72',
#     'sea_level_pressure_mean_lag72', 'wind_direction_mean_lag72',
#     'wind_speed_mean_lag72', 'air_temperature_mean_lag3',
#     'air_temperature_max_lag3',
#     'air_temperature_min_lag3', 'cloud_coverage_mean_lag3',
#     'dew_temperature_mean_lag3',
#     'precip_depth_1_hr_mean_lag3', 'sea_level_pressure_mean_lag3',
#     'wind_direction_mean_lag3', 'wind_speed_mean_lag3']

# Train model

To win in kaggle competition, how to evaluate your model is important.
What kind of cross validation strategy is suitable for this competition? This is time series data, so it is better to consider time-splitting.

However this notebook is for simple tutorial, so I will proceed with KFold splitting without shuffling, so that at least near-term data is not included in validation.

In [ ]:
def create_X_y(train_df, target_meter):
    target_train_df = train_df[train_df['meter'] == target_meter]
    target_train_df = target_train_df.merge(building_meta_df, on='building_id', how='left')
    target_train_df = target_train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
    X_train = target_train_df[feature_cols + category_cols]
    y_train = target_train_df['meter_reading_log1p'].values

    del target_train_df
    return X_train, y_train

In [ ]:
# folds = 5
seed = 666
shuffle = False
kf = sk.model_selection.KFold(n_splits=SETTINGS.model.folds, shuffle=shuffle, random_state=seed)
oof_total = 0

# Train model by each meter type

In [ ]:
target_meter = 0
X_train, y_train = create_X_y(train_df, target_meter=target_meter)
y_valid_pred_total = np.zeros(X_train.shape[0])
gc.collect()
print('target_meter', target_meter, X_train.shape)

cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
print('cat_features', cat_features)

logging.info(" *** Step 4: Build model *** ".format())

In [ ]:
def fit_lgbm(train, val, devices=(-1,), seed=None, cat_features=None, num_rounds=1500, lr=0.1, bf=0.1):
    """Train Light GBM model"""
    X_train, y_train = train
    X_valid, y_valid = val
    metric = 'l2'
    params = {'num_leaves': 31,
              'objective': 'regression',
              #               'max_depth': -1,
              'learning_rate': lr,
              "boosting": "gbdt",
              "bagging_freq": 5,
              "bagging_fraction": bf,
              "feature_fraction": 0.9,
              "metric": metric,
              #               "verbosity": -1,
              #               'reg_alpha': 0.1,
              #               'reg_lambda': 0.3
              }
    device = devices[0]
    if device == -1:
        # use cpu
        pass
    else:
        # use gpu
        print(f'using gpu device_id {device}...')
        params.update({'device': 'gpu', 'gpu_device_id': device})

    params['seed'] = seed

    early_stop = 20
    verbose_eval = 20

    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_features)
    watchlist = [d_train, d_valid]

    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=SETTINGS.model.num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)

    # predictions
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    print('best_score', model.best_score)
    log = {'train/mae': model.best_score['training']['l2'],
           'valid/mae': model.best_score['valid_1']['l2']}
    return model, y_pred_valid, log

In [ ]:
models0 = []
for train_idx, valid_idx in kf.split(X_train, y_train):
    train_data = X_train.iloc[train_idx, :], y_train[train_idx]
    valid_data = X_train.iloc[valid_idx, :], y_train[valid_idx]

    print('train', len(train_idx), 'valid', len(valid_idx))
    #     model, y_pred_valid, log = fit_cb(train_data, valid_data, cat_features=cat_features, devices=[0,])
    model, y_pred_valid, log = fit_lgbm(train_data, valid_data, cat_features=category_cols,
                                        num_rounds=SETTINGS.model.num_rounds, lr=0.05, bf=0.7)
    y_valid_pred_total[valid_idx] = y_pred_valid
    models0.append(model)
    gc.collect()
    if SETTINGS.control.debug:
        break

In [ ]:
sns.distplot(y_train)
sns.distplot(y_valid_pred_total)

oof0 = sk.metrics.mean_squared_error(y_train, y_valid_pred_total)
oof_total += oof0 * len(y_train)

del X_train, y_train
gc.collect()

In [ ]:
def plot_feature_importance(model):
    importance_df = pd.DataFrame(model.feature_importance(),
                                 index=feature_cols + category_cols,
                                 columns=['importance']).sort_values('importance')
    fig, ax = plt.subplots(figsize=(8, 8))
    importance_df.plot.barh(ax=ax)
    fig.show()

In [ ]:
target_meter = 1
X_train, y_train = create_X_y(train_df, target_meter=target_meter)
y_valid_pred_total = np.zeros(X_train.shape[0])
gc.collect()
print('target_meter', target_meter, X_train.shape)

cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
print('cat_features', cat_features)

models1 = []
for train_idx, valid_idx in kf.split(X_train, y_train):
    train_data = X_train.iloc[train_idx ,:], y_train[train_idx]
    valid_data = X_train.iloc[valid_idx ,:], y_train[valid_idx]

    print('train', len(train_idx), 'valid', len(valid_idx))
    #     model, y_pred_valid, log = fit_cb(train_data, valid_data, cat_features=cat_features, devices=[0,])
    model, y_pred_valid, log = fit_lgbm(train_data, valid_data, cat_features=category_cols, num_rounds=SETTINGS.model.num_rounds,
                                        lr=0.05, bf=0.5)
    y_valid_pred_total[valid_idx] = y_pred_valid
    models1.append(model)
    gc.collect()
    if SETTINGS.control.debug:
        break

sns.distplot(y_train)
sns.distplot(y_valid_pred_total)

oof1 = sk.metrics.mean_squared_error(y_train, y_valid_pred_total)
oof_total += oof1 * len(y_train)

del X_train, y_train
gc.collect()

In [ ]:
target_meter = 2
X_train, y_train = create_X_y(train_df, target_meter=target_meter)
y_valid_pred_total = np.zeros(X_train.shape[0])

gc.collect()
print('target_meter', target_meter, X_train.shape)

cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
print('cat_features', cat_features)

models2 = []
for train_idx, valid_idx in kf.split(X_train, y_train):
    train_data = X_train.iloc[train_idx ,:], y_train[train_idx]
    valid_data = X_train.iloc[valid_idx ,:], y_train[valid_idx]

    print('train', len(train_idx), 'valid', len(valid_idx))
    #     model, y_pred_valid, log = fit_cb(train_data, valid_data, cat_features=cat_features, devices=[0,])
    model, y_pred_valid, log = fit_lgbm(train_data, valid_data, cat_features=category_cols,
                                        num_rounds=SETTINGS.model.num_rounds, lr=0.05, bf=0.8)
    y_valid_pred_total[valid_idx] = y_pred_valid
    models2.append(model)
    gc.collect()
    if SETTINGS.control.debug:
        break

sns.distplot(y_train)
sns.distplot(y_valid_pred_total)

oof2 = sk.metrics.mean_squared_error(y_train, y_valid_pred_total)
oof_total += oof2 * len(y_train)

del X_train, y_train
gc.collect()

In [ ]:
target_meter = 3
X_train, y_train = create_X_y(train_df, target_meter=target_meter)
y_valid_pred_total = np.zeros(X_train.shape[0])

gc.collect()
print('target_meter', target_meter, X_train.shape)

cat_features = [X_train.columns.get_loc(cat_col) for cat_col in category_cols]
print('cat_features', cat_features)

models3 = []
for train_idx, valid_idx in kf.split(X_train, y_train):
    train_data = X_train.iloc[train_idx ,:], y_train[train_idx]
    valid_data = X_train.iloc[valid_idx ,:], y_train[valid_idx]

    print('train', len(train_idx), 'valid', len(valid_idx))
    #     model, y_pred_valid, log = fit_cb(train_data, valid_data, cat_features=cat_features, devices=[0,])
    model, y_pred_valid, log = fit_lgbm(train_data, valid_data, cat_features=category_cols, num_rounds=SETTINGS.model.num_rounds,
                                        lr=0.03, bf=0.9)
    y_valid_pred_total[valid_idx] = y_pred_valid
    models3.append(model)
    gc.collect()
    if SETTINGS.control.debug:
        break

sns.distplot(y_train)
sns.distplot(y_valid_pred_total)

oof3 = sk.metrics.mean_squared_error(y_train, y_valid_pred_total)
oof_total += oof3 * len(y_train)

del X_train, y_train
gc.collect()

# OOF SCOREs

In [ ]:
print ('oof score meter0 =', np.sqrt(oof0))
print ('oof score meter1 =', np.sqrt(oof1))
print ('oof score meter2 =', np.sqrt(oof2))
print ('oof score meter3 =', np.sqrt(oof3))
print ('oof score total  =', np.sqrt(oof_total / len(train_df)))

# Prediction on test data

In [ ]:
del train_df, weather_train_df, building_meta_df
gc.collect()

In [ ]:
def create_X(test_df, target_meter):
    target_test_df = test_df[test_df['meter'] == target_meter]
    target_test_df = target_test_df.merge(building_meta_df, on='building_id', how='left')
    target_test_df = target_test_df.merge(weather_test_df, on=['site_id', 'timestamp'], how='left')
    X_test = target_test_df[feature_cols + category_cols]
    return X_test

In [ ]:
def pred(X_test, models, batch_size=1000000):
    iterations = (X_test.shape[0] + batch_size -1) // batch_size
    print('iterations', iterations)

    y_test_pred_total = np.zeros(X_test.shape[0])
    for i, model in enumerate(models):
        print(f'predicting {i}-th model')
        for k in tqdm.tqdm(range(iterations)):
            y_pred_test = model.predict(X_test[k * batch_size:(k + 1) * batch_size], num_iteration=model.best_iteration)
            y_test_pred_total[k * batch_size:(k + 1) * batch_size] += y_pred_test

    y_test_pred_total /= len(models)
    return y_test_pred_total

In [ ]:
%%time
X_test = create_X(test_df, target_meter=0)
gc.collect()

y_test0 = pred(X_test, models0)

sns.distplot(y_test0)

del X_test
gc.collect()

In [ ]:
%%time
X_test = create_X(test_df, target_meter=1)
gc.collect()

y_test1 = pred(X_test, models1)
sns.distplot(y_test1)

del X_test
gc.collect()

In [ ]:
%%time
X_test = create_X(test_df, target_meter=2)
gc.collect()

y_test2 = pred(X_test, models2)
sns.distplot(y_test2)

del X_test
gc.collect()

In [ ]:
X_test = create_X(test_df, target_meter=3)
gc.collect()

y_test3 = pred(X_test, models3)
sns.distplot(y_test3)

del X_test
gc.collect()

In [ ]:
sample_submission.loc[test_df['meter'] == 0, 'meter_reading'] = np.expm1(y_test0)
sample_submission.loc[test_df['meter'] == 1, 'meter_reading'] = np.expm1(y_test1)
sample_submission.loc[test_df['meter'] == 2, 'meter_reading'] = np.expm1(y_test2)
sample_submission.loc[test_df['meter'] == 3, 'meter_reading'] = np.expm1(y_test3)

In [ ]:
sample_submission.to_csv(SETTINGS.data.path_output / 'submission.csv', index=False, float_format='%.4f')

# Replace to UCF data
if 0:
    # %%
    leak_score = 0

    leak_df = pd.read_pickle(ucf_root / 'site0.pkl')
    leak_df['meter_reading'] = leak_df.meter_reading_scraped
    leak_df.drop(['meter_reading_original', 'meter_reading_scraped'], axis=1, inplace=True)
    leak_df.fillna(0, inplace=True)
    leak_df = leak_df[leak_df.timestamp.dt.year > 2016]
    leak_df.loc[leak_df.meter_reading < 0, 'meter_reading'] = 0  # remove large negative values

    sample_submission.loc[sample_submission.meter_reading < 0, 'meter_reading'] = 0

    for bid in leak_df.building_id.unique():
        temp_df = leak_df[(leak_df.building_id == bid)]
        for m in temp_df.meter.unique():
            v0 = sample_submission.loc[(test_df.building_id == bid) & (test_df.meter == m), 'meter_reading'].values
            v1 = temp_df[temp_df.meter == m].meter_reading.values

            leak_score += sk.metrics.mean_squared_error(np.log1p(v0), np.log1p(v1)) * len(v0)

            sample_submission.loc[(test_df.building_id == bid) & (test_df.meter == m), 'meter_reading'] = temp_df[
                temp_df.meter == m].meter_reading.values

    # %%
    if not SETTINGS.control.debug:
        sample_submission.to_csv('submission_ucf_replaced.csv', index=False, float_format='%.4f')

    # %%
    sample_submission.head()

    # %%
    np.log1p(sample_submission['meter_reading']).hist()

    # %% [markdown]
    # # UCF score

    # %%
    print('UCF score = ', np.sqrt(leak_score / len(leak_df)))

    # %%
    plot_feature_importance(models0[1])

    # %%
    plot_feature_importance(models1[1])

    # %%
    plot_feature_importance(models2[1])

    # %%
    plot_feature_importance(models3[1])

    # %% [markdown]
    # # References
    #
    # These kernels inspired me to write this kernel, thank you for sharing!
    #
    #  - https://www.kaggle.com/rishabhiitbhu/ashrae-simple-eda
    #  - https://www.kaggle.com/isaienkov/simple-lightgbm
    #  - https://www.kaggle.com/ryches/simple-lgbm-solution